In [1]:
from sqlalchemy import create_engine
import pandas as pd

In [3]:
# Database Connection
db_user = 'ufnbfacj9c7u80'
db_password = 'pa129f8c5adad53ef2c90db10cce0c899f8c7bdad022cca4e85a8729b19aad68d'
db_host = 'ceq2kf3e33g245.cluster-czrs8kj4isg7.us-east-1.rds.amazonaws.com'
db_port = '5432'
db_name = 'd9f89h4ju1lleh'

connection_string = f'postgresql+psycopg2://{db_user}:{db_password}@{db_host}:{db_port}/{db_name}'
engine = create_engine(connection_string)

In [134]:
# Query
query = """
    SELECT 
        serialno,
        valp,
        hincp,
        fincp,
        yrblt,
        np,
        nr,
        bds,
        bld,
        puma_normalized,
        region
    FROM acs_pums
    WHERE
        acs_year = 2022
        AND valp > 0
        AND ten = 1
        AND valp IS NOT NULL
        AND fincp IS NOT NULL
        AND yrblt IS NOT NULL;
"""
print("*Querying database*")
df = pd.read_sql(query, engine)
print("\nProcess Complete:\n")
print(df.head(10))

*Querying database*

Process Complete:

        serialno       valp     hincp     fincp   yrblt  np   nr  bds  bld  \
0  2018HU0654721   175000.0   50000.0   50000.0  1980.0   2  0.0  3.0  2.0   
1  2018HU0654727  2100000.0  235000.0  235000.0  2000.0   2  0.0  5.0  2.0   
2  2018HU0654730   180000.0   40100.0   40100.0  1970.0   2  0.0  3.0  2.0   
3  2018HU0654829   600000.0  106400.0  106400.0  1940.0   9  0.0  3.0  2.0   
4  2018HU0654843   700000.0  315000.0  315000.0  1960.0   4  0.0  4.0  2.0   
5  2018HU0654868   500000.0   93000.0   93000.0  1950.0   5  0.0  3.0  2.0   
6  2018HU0654869   450000.0  131900.0  131900.0  1939.0   3  0.0  4.0  2.0   
7  2018HU0654903   450000.0  113000.0  113000.0  1970.0   4  0.0  3.0  2.0   
8  2018HU0654913   240000.0  170400.0  170400.0  2000.0   4  0.0  4.0  2.0   
9  2018HU0654943  1500000.0  203800.0  203800.0  1960.0   6  1.0  4.0  2.0   

   puma_normalized  region  
0               -9       4  
1               -9       4  
2             

In [136]:
# Data preparation
features = ["hincp", "fincp", "bds", "yrblt", "np", "bds", "bld", "nr"]
df = df.dropna(subset=features + ["valp"])
df = df.drop_duplicates(subset='serialno', keep='first')
df = df.astype({'bld': int, 'bds': int, 'nr': int, 'yrblt': int})

In [138]:
# Final Checks

# Check unrealistic build years
yrblt_check = df[(df['yrblt'] < 1800) | (df['yrblt'] > 2025)]
print(f"Unrealistic year built values: {len(yrblt_check)}")

# Check for zero or negative income values
zero_hincp = df[df['hincp'] <= 0]
zero_fincp = df[df['fincp'] <= 0]
print(f"Zero or negative household income entries: {len(zero_hincp)}")
print(f"Zero or negative family income entries: {len(zero_fincp)}")

# Check for zero or negative property value
zero_valp = df[df['valp'] <= 0]
print(f"Zero or negative property value entries: {len(zero_valp)}")


# Remove values
# df = df[df['yrblt'].between(1800, 2025)]
df = df[df['hincp'] > 0]
df = df[df['fincp'] > 0]
df = df[df['valp'] > 0]

Unrealistic year built values: 0
Zero or negative household income entries: 555
Zero or negative family income entries: 664
Zero or negative property value entries: 0


In [140]:
# Add potential calculated features
df['house_age'] = 2025 - df['yrblt']

In [142]:
display(df.info())

<class 'pandas.core.frame.DataFrame'>
Index: 209431 entries, 0 to 210094
Data columns (total 12 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   serialno         209431 non-null  object 
 1   valp             209431 non-null  float64
 2   hincp            209431 non-null  float64
 3   fincp            209431 non-null  float64
 4   yrblt            209431 non-null  int32  
 5   np               209431 non-null  int64  
 6   nr               209431 non-null  int32  
 7   bds              209431 non-null  int32  
 8   bld              209431 non-null  int32  
 9   puma_normalized  209431 non-null  int64  
 10  region           209431 non-null  int64  
 11  house_age        209431 non-null  int32  
dtypes: float64(3), int32(5), int64(3), object(1)
memory usage: 16.8+ MB


None

In [144]:
display(df)

,serialno,valp,hincp,fincp,yrblt,np,nr,bds,bld,puma_normalized,region,house_age
0,2018HU0654721,175000.0,50000.0,50000.0,1980,2,0,3,2,-9,4,45
1,2018HU0654727,2100000.0,235000.0,235000.0,2000,2,0,5,2,-9,4,25
2,2018HU0654730,180000.0,40100.0,40100.0,1970,2,0,3,2,-9,4,55
3,2018HU0654829,600000.0,106400.0,106400.0,1940,9,0,3,2,-9,4,85
4,2018HU0654843,700000.0,315000.0,315000.0,1960,4,0,4,2,-9,4,65
...,...,...,...,...,...,...,...,...,...,...,...,...
210090,2022HU1409970,1800000.0,91800.0,91800.0,2000,2,0,4,2,117,4,25
210091,2022HU1409978,1500000.0,116000.0,116000.0,1960,2,0,4,2,1312,4,65
210092,2022HU1409990,500000.0,177000.0,177000.0,2000,2,0,4,2,7103,4,25
210093,2022HU1410009,625000.0,42400.0,22400.0,1960,5,1,4,2,11102,4,65
